In [8]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import math
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.NRMS import NRMSModel

hparams = {
    'mode':'demo',
    'batch_size':5,
    'title_size':20,
    'his_size':20,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'value_dim':16,
    'head_num':16,
    'kernel_num':11,
    'epochs':10,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title'],
}

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = 'models/model_params/NRMS_{}_{}'.format(hparams['mode'],hparams['epochs']) +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test, mode='test')

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [ ]:
nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(device)
nrmsModel.train()

# nrmsModel.load_state_dict(torch.load(save_path))
# nrmsModel.eval()

In [ ]:
if nrmsModel.training:
    print("training...")
    loss_func = getLoss(nrmsModel)
    optimizer = optim.Adam(nrmsModel.parameters(),lr=0.001)
    nrmsModel = run_train(nrmsModel,loader_train,optimizer,loss_func,epochs=hparams['epochs'], interval=10)

In [5]:
print("evaluating...")
nrmsModel.eval()
nrmsModel.cdd_size = 1
run_eval(nrmsModel,loader_test)

0it [00:00, ?it/s]evaluating...
2191it [01:26, 25.34it/s]


{'group_auc': 0.5452, 'ndcg@5': 0.2576, 'ndcg@10': 0.3175, 'mean_mrr': 0.239}

In [9]:
nrmsModel.cdd_size = 5
torch.save(nrmsModel.state_dict(), save_path)